In [ ]:
import pandas as pd

file_path = "gpa2.csv"
df = pd.read_csv(file_path)
df

In [ ]:
def check_gpa_conversion(file_path):
    """
    Kiểm tra xem quy đổi giữa 'score10' và 'score4' trong file CSV
    có đúng theo bảng chuyển đổi GPA không.
    """

    def convert_10_to_4(score10):
        """Chuyển điểm hệ 10 sang hệ 4 theo quy tắc chuẩn."""
        if pd.isna(score10):
            return None
        elif score10 >= 8.5:
            return 4.0
        elif score10 >= 8.0:
            return 3.5
        elif score10 >= 7.0:
            return 3.0
        elif score10 >= 6.5:
            return 2.5
        elif score10 >= 5.5:
            return 2.0
        elif score10 >= 5.0:
            return 1.5
        elif score10 >= 4.0:
            return 1.0
        else:
            return 0.0

    # 1️⃣ Read CSV
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()

    if 'score10' not in df.columns or 'score4' not in df.columns:
        raise KeyError("File CSV phải có cột 'score10' và 'score4'.")

    # 2️⃣ Convert and Compare
    df['expected_score4'] = df['score10'].apply(convert_10_to_4)
    df['correct_conversion'] = df['score4'].round(1) == df['expected_score4']

    # 3️⃣ Show incorrect rows
    incorrect = df[~df['correct_conversion']]

    if incorrect.empty:
        print("✅ Tất cả các quy đổi giữa hệ 10 và hệ 4 đều CHÍNH XÁC.")
    else:
        print("⚠️ Phát hiện quy đổi sai ở các dòng sau:")
        print(incorrect[['Học kỳ', 'Mã học phần', 'Tên học phần', 'score10', 'score4', 'expected_score4']])

    # 4️⃣ Return DataFrame for further use
    return df

# --- Example usage ---
df = check_gpa_conversion(file_path)

In [ ]:
# import pandas as pd

# # Sample DataFrame
# file_path = 'gpa2.csv'
# df_origin = pd.read_csv(file_path)
# df = df_origin.copy()

# courses_to_drop = ['Bóng chuyền hơi', 'Giáo dục thể chất cơ bản', 'Bóng rổ']
# for course in courses_to_drop:
#     df = df[df['Tên học phần'] != course]

# df = df[df['validate'] != 'Không đạt']

In [ ]:
def calculate_gpa(file_path):
    """
    Đọc file CSV và tính Điểm Trung Bình Tích Lũy (TBTL)
    cho hệ 10 và hệ 4, dựa trên các cột:
    ['credits', 'score10', 'score4']
    """

    try:
        # 1. Đọc file CSV và chuẩn hóa tiêu đề cột
        df_origin = pd.read_csv(file_path)
        df = df_origin.copy()

        # Xóa các môn giáo dục thể chất vì nó không tính vào điểm tích lũy
        courses_to_drop = ['Bóng chuyền hơi', 'Giáo dục thể chất cơ bản', 'Bóng rổ']
        for course in courses_to_drop:
            df = df[df['Tên học phần'] != course]

        # loại bỏ các môn không đạt
        df = df[df['validate'] != 'Không đạt']

        # 2. Kiểm tra cột bắt buộc
        required_cols = ['credits', 'score10', 'score4']
        for col in required_cols:
            if col not in df.columns:
                raise KeyError(f"Thiếu cột bắt buộc: '{col}' trong file CSV.")

        # 3. Chuyển đổi dữ liệu sang dạng số
        df['credits'] = pd.to_numeric(df['credits'], errors='coerce')
        df['score10'] = pd.to_numeric(df['score10'], errors='coerce')
        df['score4'] = pd.to_numeric(df['score4'], errors='coerce')

        # 4. Bỏ qua các dòng không có số tín chỉ
        df = df.dropna(subset=['credits'])

        # 5. Tính điểm tích lũy
        df['Tich_luy_H10'] = df['score10'] * df['credits']
        df['Tich_luy_H4'] = df['score4'] * df['credits']

        # 6. Tổng số tín chỉ và tổng điểm
        total_credits = df['credits'].sum()
        total_points_h10 = df['Tich_luy_H10'].sum()
        total_points_h4 = df['Tich_luy_H4'].sum()

        # 7. Tính GPA
        if total_credits == 0:
            gpa_h10 = gpa_h4 = 0.0
        else:
            gpa_h10 = total_points_h10 / total_credits
            gpa_h4 = total_points_h4 / total_credits

        # 8. In kết quả
        print("--- KẾT QUẢ TÍNH TOÁN TBTL ---")
        print(f"Tổng số môn học: {len(df)}")
        print(f"Tổng số Tín chỉ (Mẫu số): {total_credits:.0f} TC")
        print("-" * 30)
        print(f"Tổng Điểm Tích Lũy (Hệ 10): {total_points_h10:.2f}")
        print(f"Điểm Trung Bình Tích Lũy (GPA Hệ 10): {gpa_h10:.4f}")
        print("-" * 30)
        print(f"Tổng Điểm Tích Lũy (Hệ 4): {total_points_h4:.2f}")
        print(f"Điểm Trung Bình Tích Lũy (GPA Hệ 4): {gpa_h4:.4f}")
        print("------------------------------")

        # 9. Trả về kết quả (phục vụ tái sử dụng)
        return {
            "total_courses": len(df),
            "total_credits": total_credits,
            "GPA_10": round(gpa_h10, 4),
            "GPA_4": round(gpa_h4, 4),
            'data': df
        }

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file tại đường dẫn: {file_path}")
    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình xử lý: {e}")


# --- Example usage ---
csv_file_path = "gpa2.csv"
rel = calculate_gpa(csv_file_path)

In [ ]:
rel['data']